In [1]:
import numpy as np

In [2]:
import mne

In [3]:
import pandas as pd

In [4]:
import json

In [5]:
%matplotlib
import matplotlib.pyplot as plt

Using matplotlib backend: Qt5Agg


In [6]:
%load_ext autoreload

%autoreload 2
import tools

In [7]:
import numpy as np

In [8]:
import itertools

In [9]:
EMPTY_ACTION = -1

In [10]:
import itertools

In [11]:
import eeg_events

In [12]:
class OfflineJoluEngagementEnvironment():
    filename = None
    iterator = None
    event = None
    
    states = ["".join(t) for t in itertools.product(*[[str(i) for i in range(0, 6)]] * 3)]
    actions = {
        0: "o",
        1: "x",
        2: "n",
        3: "*",
        4: "u",
        5: "-",
        -1: "$"
    }
    
    def __init__(self, filename):
        self.filename = filename
    
    def reset(self):
        self.iterator = eeg_events.iterate_events(self.filename)
        self.event = next(self.iterator)
        return self.states[0]
    
    def sample(self):
        event = self.event[1]
        if not event.get("collect"):
            return self.actions[event["response"]['result']["reels"]["view"][0][0]]
        else:
            return self.actions[EMPTY_ACTION]
        
    def get_state(self):
        event_tuple = self.event
        event = event_tuple[1]
        state_array = event["response"]['result']["state"]["towerStages"]
        state = "".join([str(i) for i in state_array])
        return state
    
    def get_reward(self):
        event_tuple = self.event
        event_data = event_tuple[2]
        assert event_data.shape[0] > 0
        return np.median(event_data)
        
    def step(self, action):
        event = self.event
        
        next_state = self.get_state()
        reward = self.get_reward()
        
        self.event = next(self.iterator, None)
        
        done = not self.event
        
        return next_state, reward, done, None

In [13]:
env = OfflineJoluEngagementEnvironment('test-filtered')

In [14]:
env.reset()

Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.


'000'

In [15]:
done = False
while not done:
    print(env.sample())
    state_tuple = env.step('')
    _, _, done, _ = state_tuple
    print(state_tuple)

u
('001', 5.163911581039429, False, None)
u
('002', 3.6251907348632812, False, None)
o
('113', 3.287583887577057, False, None)
$
('000', 3.373575985431671, False, None)
-
('000', 3.4239096641540527, False, None)
-
('000', 3.104528248310089, False, None)
u
('001', 3.057699680328369, False, None)
-
('001', 3.2693721055984497, False, None)
-
('001', 3.0806952118873596, False, None)
-
('001', 3.011557877063751, False, None)
-
('001', 3.154055893421173, False, None)
-
('001', 3.3953155875205994, False, None)
*
('011', 3.498696982860565, False, None)
-
('011', 3.7934980988502502, False, None)
*
('021', 3.8866488933563232, False, None)
$
('000', 3.7224859595298767, False, None)
u
('001', 3.0336636304855347, False, None)
-
('001', 3.1165533661842346, False, None)
-
('001', 3.2759527564048767, False, None)
*
('011', 3.7076988220214844, False, None)
-
('011', 3.447341799736023, False, None)
-
('011', 2.9384170174598694, False, None)
-
('011', 3.1007161736488342, False, None)
*
('021', 3.74367421

### SARSA

In [16]:
env = OfflineJoluEngagementEnvironment('test-filtered')

In [17]:
alpha = 0.15
gamma = 0.99

In [18]:
Q = {}

In [19]:
states_number = {}

In [20]:
from tqdm import tqdm_notebook

In [21]:
for i in tqdm_notebook(range(0, 100)):
    r = 0
    state = env.reset()
    action = env.sample()
    while True:
        nextstate, reward, done, _ = env.step(action)
        if done:
            break
        nextaction = env.sample()
#         print(state, action, nextstate, nextaction)
        if (state, action) not in Q:
            Q[(state, action)] = 0
            states_number[(state, action)] = []
        if (nextstate, nextaction) not in Q:
            Q[(nextstate, nextaction)] = 0
            states_number[(nextstate, nextaction)] = []
        Q[(state, action)] += alpha * (reward + gamma * Q[(nextstate, nextaction)] - Q[(state, action)])
        states_number[(state, action)].append(reward)
        action = nextaction
        state = nextstate
        r += reward

/home/anatoli/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.
Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.
Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.
Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.
Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.
Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.
Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found


Ready.
Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.
Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.
Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.
Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.
Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.
Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not found
    Range : 0 ... 108903 =      0.000 ...   425.402 secs
Ready.
Opening raw data file ./valid-data/processed/test-filtered.raw.fif...
Isotrak not

In [22]:
symbols_engagement = list(filter(lambda x: x[-1] != 0, sorted(Q.items(), key=lambda x: x[1])))

In [23]:
def calculate_reward(state):
    bells = int(state[0])
    stars = int(state[1])
    horsehoes = int(state[2])
    return joker_rewards["bell"][bells] + joker_rewards["star"][stars] + joker_rewards["horsehoe"][horsehoes]

In [24]:
joker_rewards = {
    "bell": [0, 10, 40, 100, 200, 350],
    "star": [0, 3, 8, 16, 29, 50],
    "horsehoe": [0, 1, 2, 4, 7, 10],
    "biggest": 500
}

In [25]:
symbols_engagement = [(a, calculate_reward(a), len(states_number[(a, b)]), b, c) for (a, b), c in symbols_engagement]

In [26]:
sorted(symbols_engagement, key=lambda x: x[-1])

[('001', 1, 100, 'n', 207.7319206904381),
 ('000', 0, 100, 'n', 208.02209378653012),
 ('001', 1, 100, 'o', 215.2589908474341),
 ('101', 11, 100, '-', 216.4739571458657),
 ('002', 2, 100, 'o', 216.85266557452152),
 ('100', 10, 100, 'u', 217.39600276471873),
 ('010', 3, 100, '*', 217.79541080203734),
 ('003', 4, 100, 'n', 219.08181535349155),
 ('103', 14, 100, '$', 223.8220343630791),
 ('101', 11, 100, 'u', 224.3859538879129),
 ('112', 15, 100, '$', 225.26682726897423),
 ('002', 2, 200, 'n', 225.8075985195189),
 ('020', 8, 100, 'u', 225.9268279651877),
 ('113', 17, 100, '$', 225.93516103070652),
 ('102', 12, 200, '-', 226.33468291115915),
 ('000', 0, 100, 'x', 226.56912673301028),
 ('003', 4, 100, '$', 226.73268541436482),
 ('101', 11, 100, '$', 226.7859053028706),
 ('021', 9, 100, 'u', 226.96177645665568),
 ('003', 4, 500, '-', 227.0642671030742),
 ('013', 7, 100, 'x', 228.49989414598076),
 ('002', 2, 700, 'u', 228.77805736676376),
 ('012', 5, 100, 'x', 229.6920117156229),
 ('003', 4, 2

Пример - состояние 013. Вовлеченность существенно больше, если игрок находится в состоянии 0. Что это за состояние?

Как меняются приоритеты игрока в состояниях 0 и 2

In [27]:
def increase_state(s, i):
    return s[:i] + chr(ord(s[i]) + 1) + s[i+1:]

In [28]:
def get_transition(s, a):
    if a == '-':
        return s
    if a in ['$', 'x']:
        return '000'
    if a == 'u':
        return increase_state(s, 2)
    if a == 'n':
        return increase_state(s, 0)
    if a == '*':
        return increase_state(s, 1)
    if a == 'o':
        ns = increase_state(s, 0)
        ns = increase_state(ns, 1)
        ns = increase_state(ns, 2)
        return ns

In [29]:
get_transition("000", "o")

'111'

In [30]:
rewards = {}

In [31]:
for key, value in list(Q.items()):
    if not value:
        del Q[key]

In [32]:
V = {} # V - minimum value
for (s, a), value in Q.items():
    if s not in V:
        V[s] = max(Q.values())
    if value < V[s]:
        V[s] = value

In [33]:
sorted(V.items(), key=lambda x: x[1])

[('001', 207.7319206904381),
 ('000', 208.02209378653012),
 ('101', 216.4739571458657),
 ('002', 216.85266557452152),
 ('100', 217.39600276471873),
 ('010', 217.79541080203734),
 ('003', 219.08181535349155),
 ('103', 223.8220343630791),
 ('112', 225.26682726897423),
 ('020', 225.9268279651877),
 ('113', 225.93516103070652),
 ('102', 226.33468291115915),
 ('021', 226.96177645665568),
 ('013', 228.49989414598076),
 ('012', 229.6920117156229),
 ('011', 230.27595966425463),
 ('004', 232.26369170275083),
 ('014', 232.7802174791002),
 ('022', 234.65267440467227)]

In [34]:
for (s, a), value in Q.items():
    ns = get_transition(s, a)
    if (s, a) not in rewards:
        rewards[(s, a)] = 0
    rewards[(s, a)] = Q[(s, a)] - V[ns] / gamma

In [35]:
sorted(rewards.items(), key=lambda x: x[1])

[(('001', 'o'), -12.283258919206531),
 (('000', 'n'), -11.569828198034259),
 (('002', 'o'), -11.364668799929518),
 (('001', 'n'), -10.928642083163624),
 (('010', '*'), -10.413506334515887),
 (('021', 'u'), -10.061126982407217),
 (('003', 'n'), -7.001047639517651),
 (('013', 'u'), -4.310553315217163),
 (('101', 'u'), -4.234937941540778),
 (('003', 'u'), -4.015026519675217),
 (('020', 'u'), -3.327491687999867),
 (('012', '*'), -3.0243009883089655),
 (('002', 'n'), -2.813293309934778),
 (('102', '-'), -2.286208918294534),
 (('101', '-'), -2.1866056277360144),
 (('100', 'u'), -1.2645600088829951),
 (('003', '*'), -1.0925792563222672),
 (('013', '-'), -0.8280317606036078),
 (('011', '-'), -0.214772555590514),
 (('001', '*'), 0.09147755168385174),
 (('010', 'u'), 0.45887710761931544),
 (('011', '*'), 1.0216400110670634),
 (('012', 'u'), 1.2598165950627163),
 (('011', 'u'), 1.5266508881189793),
 (('002', '*'), 1.6449925208023046),
 (('012', '-'), 2.127706509874457),
 (('003', '-'), 5.76950411